# Building Blocks
## The important pieces to analyze (a lot of) data

# RDD - Resiliant Distributed Dataset
Spark's basic data collection is an RDD:
* *Resiliant* - Any piece that is lost can be regenerated
* *Distributed* - Pieces are scattered across multiple nodes
* *Dataset* - A large number of "items" either generated from files or by translating other RDDs

No restriction that each "item" in an RDD have the same elements, somewhat low-level for many uses

# DataFrame
DataFrames are specialized RDDs that organize their data in rows and columns, where every row has the same columns. This is similar to CMS datasets where each event in Data/MC has the same branches.

The layout of a DataFrame is more limited than a plain RDD, but this limitation allows Spark's query optimizer to speed up or even elide portions of results.

# Outline
The tools are different, but the overall workflow is familiar:

1. Import datasets from disk
2. Apply transformations
  * Perform cuts
  * Produce derived values
3. Aggregate and report to produce tables/plots

Let's first demonstrate the tools and concepts with a small, non-CMS dataset.

For our first example, we will use the US Government's records of all flights in the US during 2015 (found [here](http://stat-computing.org/dataexpo/)). It's a small dataset that Spark is ridiculously overpowered for, but its size lets us experiment instantaneously. 

## Setup
Before we do anything with Spark, we must create a `SparkSession` like you did in the pre-exercise.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder \
            .appName("10-airline-data") \
            .config("spark.jars.packages", "org.diana-hep:histogrammar-sparksql_2.11:1.0.4") \
            .getOrCreate()

In [ ]:
csvPath = "hdfs://cmshdfs/tmp/2005.csv.bz2"
airline = spark.read \
            .option("header","true") \
            .option("inferSchema", "true") \
            .csv(csvPath) \
            .sort("Month", "DayofMonth") \
            .withColumnRenamed("UniqueCarrier", "Carrier")

In [ ]:
airline.printSchema()

In [ ]:
airline = airline.withColumn("DepTime", airline["DepTime"].cast(IntegerType()))\
                    .withColumn("ArrTime", airline["ArrTime"].cast(IntegerType()))\
                    .withColumn("DepDelay", airline["DepDelay"].cast(IntegerType()))\
                    .withColumn("ArrDelay", airline["ArrDelay"].cast(IntegerType()))

In [ ]:
airline.printSchema()

In [ ]:
airline.take(1)

In [ ]:
airline.show(1)

In [ ]:
# https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html?highlight=cache#pyspark.sql.DataFrame.drop
airline = airline.drop("CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay")
airline.show(1)

In [ ]:
trimmedAirline = airline.select("Month", "DayOfMonth", "DayOfWeek",
                                "DepTime", "ArrDelay", "DepDelay",
                                "TaxiIn", "TaxiOut", "Origin",
                                "Dest", "Distance", "Carrier",
                                "FlightNum", "Cancelled")

In [ ]:
trimmedAirline.show(5)

In [ ]:
# Do some filtering
trimmedAirline.where(trimmedAirline.Carrier != "UA").show(5)

# What about a more complicated expression?
```python
trimmedAirline.where(trimmedAirline.Month == 1 and trimmedAirline.DayOfMonth == 8).show(5)
```

uh oh..
```
ValueError: Cannot convert column into bool: please use '&' for 'and'
```
Let's try that....
```python
trimmedAirline.where(trimmedAirline.Month == 1 & trimmedAirline.DayOfMonth == 8).show(5)
```

The previous cell complains:
```
Method and([class java.lang.Integer]) does not exist
```
The "&" operator has weird operator precidence, so the previous line:
```python
trimmedAirline.where(trimmedAirline.Month == 1 & trimmedAirline.DayOfMonth == 8).show(5)
```
Is interpreted as
```python
trimmedAirline.where(trimmedAirline.Month == (1 & trimmedAirline.DayOfMonth) == 8).show(5)
```
Add explicit parenthesis to force what we really mean:
```python
trimmedAirline.where((trimmedAirline.Month == 1) & (trimmedAirline.DayOfMonth == 8)).show(5)
```

# "SQL string" format
This is a lot easier for complicated expressions
```python
trimmedAirline.where("Month == 1 and DayOfMonth == 8").show(5)
```

In [ ]:
trimmedAirline.where("Month == 1 and Origin == 'ORD' and Dest == 'EWR'").count()

In [ ]:
trimmedAirline.where("Month == 1 and DayOfMonth == 9 and Origin == 'ORD' and Dest == 'EWR'").count()

In [ ]:
trimmedAirline.where("""Month == 1 AND DayOfMonth == 9 AND 
                        ((Origin == 'ORD' AND Dest == 'EWR') OR
                         (Origin == 'EWR' AND Dest == 'ORD'))""").count()

# Rename columns

In [ ]:
renamed = trimmedAirline.withColumnRenamed("DayOfMonth", "Day")
renamed.show()

## Produce derived values

In [ ]:
# Add a simple column for "How much time was 'made up in the air'"
makeupTime = renamed.withColumn("Makeup", (renamed.ArrDelay - renamed.DepDelay).cast("integer"))
makeupTime.show(5)

In [ ]:
# What if we want something thats more complicated than just simple arithmetic?
# We can make a UDF - User Defined Function

# Spark will execute the following function for each row. You can put arbitrary python
# code here (with some restrictions)
import datetime
def dayOfYear(month, day):
    """
    Given a month and day, return the day of year.
        (e.g. Jan 1st is day "0")
    """
    return datetime.date(2005,month,day).timetuple()[7] - 1

from pyspark.sql.functions import udf
from pyspark.sql.types import *
dayUDF = udf(dayOfYear, IntegerType())

withDay = makeupTime.withColumn("DayCount", dayUDF("Month", "Day"))
withDay.show(5)

In [ ]:
# Suppose you wanted to correct Jet energies and make corresponding changes
# to the MET of the event. It would be difficult and inefficient to run two UDFs.
# Fortunately, we aren't limited to a single output column. We can pass back a
# structure to spark, and it will expand each memeber into a column

dateSchema = StructType([
    StructField("Date", DateType(), False),
    StructField("Count", IntegerType(), False),
])

def dateUDF(month, day):
    """
    Given a month and day, return the day of year and the 
        (e.g. Jan 1st is day "0")
    """
    dateObj = datetime.date(2005,month,day)
    dayCount = dateObj.timetuple()[7] - 1
    return (dateObj, dayCount)

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
dayUDF = udf(dateUDF, dateSchema)

withDayNested = makeupTime.withColumn("udf2", dayUDF("Month", "Day"))
withDayNested.show(5)

In [ ]:
withDayFlat = withDayNested.select("udf2.Date", "udf2.Count")
withDayFlat.show(5)

In [ ]:
data = withDayNested \
                .withColumn("FlatDate", withDayNested.udf2.Date) \
                .withColumn("FlatCount", withDayNested.udf2.Count) \
                .drop(withDayNested.udf2)
data.show(5)

# Aggregate and Report
Once we've loaded our data, done some filters and added some generated values, we'll want to aggregate the information so we can report values and/or make plots.

In [ ]:
# How many flights are cancelled on each day of the week
days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
cancelledPerDay = []
for i in range(len(days)):
    cancelledPerDay.append(data.where("DayOfWeek == %i and Cancelled == 1" % (i + 1)).count())

In [ ]:
for i in zip(days, cancelledPerDay):
    print "%s: %i" % i

### Load matplotlib and histogrammar
For simple queries, a combination of `select` and `count` are sufficient, but more advanced queries need additional packages. Run the following snippet in the next cell to load Matplotlib and Histogrammar

```python
%matplotlib inline
import matplotlib.pyplot as plt
import histogrammar as hg
import histogrammar.sparksql
```

### Decorate our DataFrame
With Histogrammar loaded, we can make our DataFrame "Histogrammar-aware" by executing the following:
    
```python
    hg.sparksql.addMethods(data)
```

As you can probably guess, `addMethods` adds some magic methods to allow our Dataframe to be processed by Histogrammar

In [ ]:
hg.sparksql.addMethods(data)

### Plot a single variable
At this point, we've got all the piece we need to actually extract some meaningful information from our dataset. Let's run the simplest-possible plotting in the next cell with:

```python
%%timeit -n3
h1 = data.Bin(100, 0, 200, data['ArrDelay'])
ax = h1.plot.matplotlib(name="Arrival Delay (min)")
```

The `%%timeit` magic command will execute the cell 3 times and report the best iteration

### What just happened?
When the previous cell was executed, Spark did the following using several distributed workers:
1. Load the data from the CSV and divide into a number of `partitions`
2. Perform each transformation on the partitions
3. Histogrammar fills a histogram per-partition
4. Histogrammar uses Spark to perform a distributed reduce/aggregation
5. The resulting histogram is passed to Matplotlib to be plotted

### Why is this useful?
* Spark hides a lot of the complexity
    * Splitting tasks
    * Retrying failed jobs
* Spark can optimize away unwanted computation
* Spark can aggressively cache intermediate results
* Distributed reduces are fast!

# More examples
Now that we've plotted a single variable, let's do some more complicated aggregations and plots.

Let's make a histogram of the departure delay, following the template from before

In [ ]:
h2 = data.Bin(100, 0, 200, data['DepDelay'])
ax = h2.plot.matplotlib(name="Departure Delay (min)")

What we've effectively done in the two previous cells is make two histograms (`h1` and `h2`), then filled them independently. In the background, Spark performed two separate aggregations. Histogrammar instead lets us combine the histograms together and perform a single aggregation.

In [ ]:
# Note only the outer UntypedLabel connects to data. The
# internal Bin objects connect to histogrammar. The outer
# container will pass the elements in
h3 = data.UntypedLabel(arrdelay=hg.Bin(100, 0, 200, data['ArrDelay']),
                       depdelay=hg.Bin(100, 0, 200, data['DepDelay']),)

In [ ]:
dd = h3.get('depdelay').plot.matplotlib(name="Departure Delay (min)")

In [ ]:
ad = h3.get('arrdelay').plot.matplotlib(name="Arrival Delay (min)")

## Multiple plots
It can be helpful to show multiple plots in a single cell

In [ ]:
plt.subplot(1, 2, 1)
dd = h3.get('depdelay').plot.matplotlib(name="Departure Delay (min)")
plt.subplot(1, 2, 2)
ad = h3.get('arrdelay').plot.matplotlib(name="Arrival Delay (min)")

## 2D histograms
Implicit in the `Bin()` definition we've seen so far is the value we want to fill. By default, the value is filled by `Count()`, which effectively adds a one into the appropriate bin.
```python
h4 = data.Bin(100, 0, 200, data['DepDelay'], value=hg.Count())
```
We can make a 2D histogram by replacing `Count()` with a 1D histogram
```python
h4 = data.Bin(100, 0, 200, data['DepDelay'],
              value=hg.Bin(100, 0, 200, data['ArrDelay']))
```

## Selecting a subset of events
The `Select()` function lets us extract a subset of events
```python
h5 = data.UntypedLabel(
        ua=hg.Select(data['Carrier'] == "UA", 
                         hg.Bin(100, 0, 200, data['ArrDelay'])),
        aa=hg.Select(data['Carrier'] == "AA", 
                         hg.Bin(100, 0, 200, data['ArrDelay']))
    )
```

## Plotting two histograms simultaneously
```python
plt.xlabel('Minutes Delayed')
h5.get('ua').plot.matplotlib(alpha=0.4, label='United')
h5.get('aa').plot.matplotlib(alpha=0.4, label='American')
plt.legend(loc='upper right')
plt.title('Delays by airline')
```

## Other aggregation operators
Histogrammar provides a [number](http://histogrammar.org/python/1.0.9/) of additional aggregation operators. For instance, we can get the average arrival delay like the following.
```python
h6 = data.UntypedLabel(
        ua=hg.Select(data['Carrier'] == "UA", 
                         hg.Average(data['ArrDelay'])),
        aa=hg.Select(data['Carrier'] == "AA", 
                         hg.Average(data['ArrDelay']))
    )
print h6.get('ua').cut.mean
```                               

# Your turn!
Before we move to a CMS dataset, use what you've learned so far to count the number of cancelled flights for each month. Do you notice anything odd?

# Other ideas
If you've got extra time, answer the following:
1. What are the 10 most popular routes?
2. What airports are the most delayed outbound? Inbound?
3. Show the trend of "mins made up" vs. distance.
4. What airline has the lowest average delay?

# Summary
We looked all US flights in 2005 and studied their on-time arrival statistics. We used Spark, Histogrammar, and Matplotlib to:
1. Import datasets from disk
2. Apply transformations
  * Perform cuts
  * Produce derived values
3. Aggregate and report to produce tables/plots

Next, we'll apply these lessons to CMS data.

# Don't forget to clean up!
Once you've completed this section and you're satisfied with the results, select "File->Close and Halt" from the Jupyter window. Spark is memory intensive, so it's good to remember to close the notebook when you're done. (Closing the notebook terminates the Spark process).